## 1. [Search API](https://developers.google.com/youtube/v3/docs/search/list?apix_params=%7B%22part%22%3A%22snippet%22%2C%22channelId%22%3A%22UCw0wD0BctMf0cUHiz_jh36Q%22%7D) 를 이용해  channel의 동영상 리스트 가져오기

In [1]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = [ "https://www.googleapis.com/auth/youtube",
"https://www.googleapis.com/auth/youtube.force-ssl",
"https://www.googleapis.com/auth/youtube.readonly",
"https://www.googleapis.com/auth/youtubepartner"]

### Google OAuth 2.0 등록

In [2]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../secret_key/client_secret.json"
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=699806263406-koq1jouhe37k38dvvshpu98ib9m5o53g.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutubepartner&state=XwvOl6pDa71GFyscjLExdUs9Lz4ojP&prompt=consent&access_type=offline
Enter the authorization code: 4/yQFMK5chAer-zBwpk0z7Bdbxc-Ln6sUp1N_Tu_X17ZrUZiQcfB4oPw0


In [3]:
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

## 유튜버의 업로드 동영상 리스트

In [9]:
# channelId: N잡하는 허대리
search_response = youtube.search().list(
    part="id,snippet",
    channelId='UCw0wD0BctMf0cUHiz_jh36Q',
    maxResults=50,
    pageToken='',
  ).execute()


# print(response)

In [27]:
def get_videos(channelId):
    videos = []
    next_page_token = ''
    
    i = 0
    maxResults=50
    while(True):
        print(i * maxResults, ':', channelId)
        search_response = youtube.search().list(
            part="id,snippet",
            channelId=channelId,
            maxResults=maxResults,
            pageToken=next_page_token,
          ).execute()
        videos += search_response['items']
        
        if 'nextPageToken' not in search_response.keys():
            break
        next_page_token = search_response['nextPageToken']
    return videos

In [33]:
channel_videos = get_videos('UCw0wD0BctMf0cUHiz_jh36Q')

0 : UCw0wD0BctMf0cUHiz_jh36Q


In [38]:
videoTitle = []
videoId = []

for channel_video in channel_videos:
    if channel_video["id"]["kind"] == "youtube#video":
        videoTitle.append(channel_video["snippet"]["title"])
        videoId.append(channel_video["id"]["videoId"])

## Videos DataFrame

In [43]:
import pandas as pd

In [54]:
def get_channel_video_df(channelId):
    get_videos(channelId)

    videoTitle = []
    videoId = []

    for channel_video in channel_videos:
        if channel_video["id"]["kind"] == "youtube#video":
            videoTitle.append(channel_video["snippet"]["title"])
            videoId.append(channel_video["id"]["videoId"])
    
    df_videos = pd.DataFrame({'videoTitle':videoTitle,'videoId': videoId})
    df_videos['channelId'] = channelId
    
    return df_videos

In [56]:
df_videos = get_channel_video_df(channelId)

0 : UCw0wD0BctMf0cUHiz_jh36Q


## 2. [Videos API]() 를 이용해 해당 Video의 좋아요, 조회수 등 디테일 정보 가져오기
